In [1]:
import pandas as pd
import numpy as np
import duckdb
import time
from pathlib import Path
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

In [2]:
%run ./common_utils.ipynb

Importing following functions: save_df(), construct_df_roster(), construct_df_teams(), concatenate_all_files()


In [72]:
con = duckdb.connect(database=':memory:')

# Allocate all rosters.csv files
save_location = "../src/injuries"
directory_path = Path(save_location)
file_paths = [entry for entry in directory_path.iterdir() if entry.is_file()]
file_names = [file.name for file in file_paths]
df = pd.DataFrame()
for i in file_names:
    df_temp = pd.read_csv(save_location + "/" + i)
    df = pd.concat([df, df_temp])
    
df = df[(df.game_type == 'REG') & (df.position.isin(['FB', 'RB']))]

# Make team names consistent with team_info_xref table
team_nm_fixes = [('GB', 'GNB'), ('KC', 'KAN'), ('LA', 'LAR'), ('LV', 'LVR'), ('NE', 'NWE'), ('NO', 'NOR'),
                 ('SD', 'SDG'), ('SF', 'SFO'), ('TB', 'TAM')]
for wrong_nm, right_nm in team_nm_fixes:
    df['team'] = np.where(df.team == wrong_nm, right_nm, df.team)

df_gmlog = pd.read_csv("../tables/players_gamelogs/players_gamelogs.csv")
df_gmlog = df_gmlog[(df_gmlog.Season_type == 'DNP')]

df = con.execute("""SELECT df_gmlog.*, df.* FROM df JOIN df_gmlog ON df.full_name = df_gmlog.Player AND df.team = df_gmlog.Team
                    AND df.season = df_gmlog.Season AND df.week = df_gmlog.Week""").fetchdf()

display(df)
# df.to_csv("../tables/test.csv", index=False)

,CarGm,Team,Player,pfr_id,Date,Season,Week,Season_type,GS,RushAtt,RushYds,RushTD,Tgt,Rec,RecYds,RecTD,OffSnp,STSnp,season_1,game_type,team_1,week_1,gsis_id,position,full_name,first_name,last_name,report_primary_injury,report_secondary_injury,report_status,practice_primary_injury,practice_secondary_injury,practice_status,date_modified
0,0,CLE,Jerome Harrison,HarrJe00,2009-09-13,2009,1,DNP,0,0,0,0,0,0,0,0,0,0,2009,REG,CLE,1,00-0024359,RB,Jerome Harrison,Jerome,Harrison,Knee,None,Questionable,Knee,None,Limited Participation in Practice,None
1,0,GNB,Brandon Jackson,JackBr00,2009-09-13,2009,1,DNP,0,0,0,0,0,0,0,0,0,0,2009,REG,GNB,1,00-0025450,RB,Brandon Jackson,Brandon,Jackson,Ankle,None,Doubtful,Ankle,None,Did Not Participate In Practice,None
2,0,NOR,Pierre Thomas,ThomPi00,2009-09-13,2009,1,DNP,0,0,0,0,0,0,0,0,0,0,2009,REG,NOR,1,00-0024813,RB,Pierre Thomas,Pierre,Thomas,Knee,None,Out,Knee,None,Did Not Participate In Practice,None
3,0,NYJ,Shonn Greene,GreeSh00,2009-09-13,2009,1,DNP,0,0,0,0,0,0,0,0,0,0,2009,REG,NYJ,1,00-0026899,RB,Shonn Greene,Shonn,Greene,Ribs,None,Probable,Ribs,None,Full Participation in Practice,None
4,0,OAK,Justin Fargas,FargJu00,2009-09-14,2009,1,DNP,0,0,0,0,0,0,0,0,0,0,2009,REG,OAK,1,00-0022093,RB,Justin Fargas,Justin,Fargas,Hamstring,None,Questionable,Hamstring,None,Limited Participation in Practice,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1434,0,NOR,Alvin Kamara,KamaAl00,2025-01-05,2024,18,DNP,0,0,0,0,0,0,0,0,0,0,2024,REG,NOR,18,00-0033906,RB,Alvin Kamara,Alvin,Kamara,Groin,None,Doubtful,Groin,None,Did Not Participate In Practice,2025-01-03T19:59:19Z
1435,0,PHI,Saquon Barkley,BarkSa00,2025-01-05,2024,18,DNP,0,0,0,0,0,0,0,0,0,0,2024,REG,PHI,18,00-0034844,RB,Saquon Barkley,Saquon,Barkley,Not injury related - coach's decision,None,Doubtful,Not injury related - resting player,None,Limited Participation in Practice,2025-01-03T19:36:30Z
1436,0,LAR,Kyren Williams,WillKy02,2023-10-22,2023,7,DNP,0,0,0,0,0,0,0,0,0,0,2023,REG,LAR,7,00-0037840,RB,Kyren Williams,Kyren,Williams,Ankle,None,Out,Ankle,None,Did Not Participate In Practice,2023-10-20T19:56:00Z
1437,0,LAR,Kyren Williams,WillKy02,2024-01-07,2023,18,DNP,0,0,0,0,0,0,0,0,0,0,2023,REG,LAR,18,00-0037840,RB,Kyren Williams,Kyren,Williams,Coach's decision,None,Doubtful,Coach's decision,None,Full Participation in Practice,2024-01-05T20:39:56Z


In [ ]:
# # Actual Injuries that I will count
# query = injuries_df[(injuries_df['position'] == 'RB') & (injuries_df['full_name'] == 'Kyren Williams')
#                     & (~injuries_df['report_primary_injury'].str.contains("Coach|Decision|Not Injury|Non Injury|COVID|Personal|Non-Football Illness|Non Football Illness|non football injury", case=False, na=False))].dropna(subset=['report_status'])
# query = query[['season', 'game_type', 'team', 'week', 'full_name', 'report_primary_injury', 'report_secondary_injury', 'report_status']]
# for i in ['season', 'week']:
#     query[i] = query[i].astype(int)
# display(query)

# ##############################################################################################################################################################################

# display(pd.read_csv("../tables/players_gamelog/Kyren Williams_gamelog.csv").query("Season_type == 'DNP'"))

# ##############################################################################################################################################################################

# # Non injuries leaves (will stay as DNP, but will be logged)
# query = injuries_df[(injuries_df['position'] == 'RB') & (injuries_df['full_name'] == 'Kyren Williams')
#                     & (injuries_df['report_primary_injury'].str.contains("Coach|Decision|Not Injury|Non Injury|COVID|Personal|Non-Football Illness|Non Football Illness|non football injury", case=False))].dropna(subset=['report_status'])
# query = query[['season', 'game_type', 'team', 'week', 'full_name', 'report_primary_injury', 'report_secondary_injury', 'report_status']]
# for i in ['season', 'week']:
#     query[i] = query[i].astype(int)
# display(query)

In [ ]:
# # Make function that checks consecutive DNP rows, and flags when player has big injury

# lst = [4, 5, 6, 9, 10, 11, 13, 14]
# lst.sort(reverse=True)
# print(lst)

# injury_list = []
# sub_list = []
# big_injury = False
# for i in lst:
#     if sub_list == []:
#         sub_list.append(i)
#         continue
#     prev_num = sub_list.pop()
#     subtract = prev_num - i
#     print(subtract)
#     sub_list.append(i)
#     if subtract == 1:
#         injury_list.append(subtract)
#     else:
#         injury_list.clear()

#     if len(injury_list) >= 3:
#         big_injury = True        
        
# if big_injury == True:
#     print('player had big injury this year')
# else:
#     print('player had no big injuries')